In [1]:
import requests
import matplotlib
import geopandas as gpd

In [2]:
BASE_URL = "https://s3.amazonaws.com/tripdata/"
month = "202412-citibike-tripdata.zip"
latest_data = requests.get(BASE_URL + month)

In [22]:
def safe_decimal(x):
    try:
        return Decimal(str(x).strip())
    except:
        return None



In [23]:
from decimal import Decimal
from io import BytesIO
import pandas as pd
from zipfile import ZipFile

# Read in a zipped CSV HTML response

zip_data = BytesIO(latest_data.content)
csv_filename = None
all_data = pd.DataFrame()
# Open the zip file
with ZipFile(zip_data) as zip_file:
    for name in zip_file.namelist():
        print(name)
        if name.endswith(".csv"):
            csv_filename = name
            with zip_file.open(name) as csv_file:
                table = pd.read_csv(
                    csv_file,
                    dtype={"start_station_id": str, "end_station_id": str},
                    converters={"start_lat": safe_decimal, "start_lng": safe_decimal, "end_lat": safe_decimal, "end_lng": safe_decimal},
                )
                all_data = pd.concat([all_data, table])


202412-citibike-tripdata_1.csv
202412-citibike-tripdata_3.csv
202412-citibike-tripdata_2.csv


In [24]:
def clean_data(data:pd.DataFrame):
    return data.dropna(subset=["start_lat", "start_lng", "end_lat", "end_lng", "start_station_id","end_station_id"])
    

In [25]:
data = clean_data(all_data)

In [31]:
grouped_data = data.groupby('start_station_id').agg({'start_lat': lambda x: x.value_counts().index[0], 'start_lng': lambda x: x.value_counts().index[0], 'start_station_name': 'first'})

In [32]:
import geopandas as gpd
grouped_data_df = gpd.GeoDataFrame(grouped_data, geometry=gpd.points_from_xy(grouped_data.start_lng, grouped_data.start_lat))

In [35]:
grouped_data_df.start_station_name.value_counts()

start_station_name
Seneca Ave & Irvine St               2
W 35 St & 9 Ave                      2
67 St & Erik Pl                      1
Central Park S & Grand Army Plaza    1
E 63 St & 3 Ave                      1
                                    ..
Flushing Ave & Woodward Ave          1
Manhattan Ave & Devoe St             1
Amory Ct & Metropolitan Ave          1
Vesey St & Greenwich St              1
Shop Morgan                          1
Name: count, Length: 2138, dtype: int64

In [40]:
grouped_data_df[grouped_data_df.start_station_name == 'W 35 St & 9 Ave']

,start_lat,start_lng,start_station_name,geometry
start_station_id,,,,
6569.09,40.75414519263519,-73.9960889518261,W 35 St & 9 Ave,POINT (-73.99609 40.75415)
6569.09_,40.75425585988168,-73.99645123326621,W 35 St & 9 Ave,POINT (-73.99645 40.75426)


In [42]:
all_data[all_data.start_station_id == '6569.09']

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
182395,FA8A68C0005DFB5F,electric_bike,2024-12-06 17:02:42.476,2024-12-06 17:11:31.025,W 35 St & 9 Ave,6569.09,E 30 St & Park Ave S,6206.08,40.75414519263519,-73.9960889518261,40.74444921,-73.98303529,member
182405,707A177D891E5854,electric_bike,2024-12-13 22:27:13.197,2024-12-13 22:46:56.090,W 35 St & 9 Ave,6569.09,E 56 St & 3 Ave,6691.11,40.75414519263519,-73.9960889518261,40.75934501,-73.96759673,casual
182436,4A06DE3D8066BB22,classic_bike,2024-12-12 19:13:28.558,2024-12-12 19:43:25.303,W 35 St & 9 Ave,6569.09,Broadway & Berry St,5164.05,40.75414519263519,-73.9960889518261,40.71036129922581,-73.96530389785767,member
184482,5B016A0A84D9CB6C,electric_bike,2024-12-12 17:34:29.058,2024-12-12 17:53:11.754,W 35 St & 9 Ave,6569.09,W 87 St & West End Ave,7484.05,40.75414519263519,-73.9960889518261,40.789622,-73.97757,member
184493,68CCB3FE4CE8B14E,electric_bike,2024-12-06 09:51:13.101,2024-12-06 10:02:06.215,W 35 St & 9 Ave,6569.09,E 30 St & Park Ave S,6206.08,40.75414519263519,-73.9960889518261,40.74444921,-73.98303529,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995780,FFAC3449053F6844,electric_bike,2024-12-30 12:46:37.034,2024-12-30 12:55:31.680,W 35 St & 9 Ave,6569.09,Broadway & W 58 St,6948.10,40.75414519263519,-73.9960889518261,40.76695317,-73.98169333,member
995809,9C1DEC9C9847C2E6,electric_bike,2024-12-27 17:23:47.485,2024-12-27 17:32:36.890,W 35 St & 9 Ave,6569.09,Broadway & W 58 St,6948.10,40.75414519263519,-73.9960889518261,40.76695317,-73.98169333,member
999203,9215A4799C3660F2,electric_bike,2024-12-15 23:14:09.657,2024-12-15 23:18:48.669,W 35 St & 9 Ave,6569.09,W 44 St & 11 Ave,6756.05,40.75414519263519,-73.9960889518261,40.762009,-73.996975,member
999493,7AD2C6842F934FAC,electric_bike,2024-12-20 15:59:32.804,2024-12-20 16:03:57.803,W 35 St & 9 Ave,6569.09,9 Ave & W 45 St,6717.06,40.75414519263519,-73.9960889518261,40.76019252,-73.9912551,casual


In [41]:
all_data[all_data.start_station_id == '6569.09_']

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
227181,8255B357C76175FF,classic_bike,2024-12-12 21:18:59.221,2024-12-12 21:29:31.003,W 35 St & 9 Ave,6569.09_,W 25 St & 6 Ave,6215.04,40.75425585988168,-73.99645123326621,40.74395411,-73.99144871,member
247606,56D988CA949781F1,electric_bike,2024-12-09 09:44:55.205,2024-12-09 09:50:32.918,W 35 St & 9 Ave,6569.09_,Broadway & W 48 St,6708.04,40.75425585988168,-73.99645123326621,40.76017739537783,-73.98486793041229,member
251277,2BEE4B5BA6ED5319,electric_bike,2024-12-14 15:24:12.287,2024-12-14 15:41:55.834,W 35 St & 9 Ave,6569.09_,University Pl & E 14 St,5905.14,40.75425585988168,-73.99645123326621,40.734814,-73.992085,member
279224,2BA31CFF4E980EEE,electric_bike,2024-12-09 18:28:33.427,2024-12-09 18:54:36.495,W 35 St & 9 Ave,6569.09_,E 58 St & 1 Ave (NW Corner),6723.1,40.75425585988168,-73.99645123326621,40.759125,-73.962658,member
310426,92540C91DAAD6BD5,classic_bike,2024-12-08 12:25:30.006,2024-12-08 12:39:18.312,W 35 St & 9 Ave,6569.09_,West End Ave & W 60 St,7059.08,40.75425585988168,-73.99645123326621,40.77237,-73.99005,member
324811,447EF34CA51E8972,classic_bike,2024-12-02 18:36:42.801,2024-12-02 18:38:27.896,W 35 St & 9 Ave,6569.09_,W 33 St & 10 Ave,6492.04,40.75425585988168,-73.99645123326621,40.753773368360385,-73.99937599897385,member
377922,11CBDCBE2A61F923,electric_bike,2024-12-02 18:16:12.354,2024-12-02 18:43:14.769,W 35 St & 9 Ave,6569.09_,E 60 St & York Ave,6682.08,40.75425585988168,-73.99645123326621,40.759107,-73.959223,member
440744,A03C81C8FCFA4A07,electric_bike,2024-12-09 06:57:33.012,2024-12-09 07:02:57.499,W 35 St & 9 Ave,6569.09_,W 22 St & 8 Ave,6224.03,40.75425585988168,-73.99645123326621,40.74475148,-73.99915362,casual
441947,10A4AF5F08DB2C28,electric_bike,2024-12-09 07:44:45.994,2024-12-09 07:52:14.018,W 35 St & 9 Ave,6569.09_,W 51 St & 6 Ave,6740.1,40.75425585988168,-73.99645123326621,40.7606597,-73.98042047,member
442581,1D7C54295FCDA9AA,classic_bike,2024-12-02 07:00:41.601,2024-12-02 07:01:49.201,W 35 St & 9 Ave,6569.09_,9 Ave & W 33 St,6492.08,40.75425585988168,-73.99645123326621,40.752568,-73.996765,member


In [33]:
grouped_data_df.to_geojson()

AttributeError: 'GeoDataFrame' object has no attribute 'to_geojson'